- Dataset? Training data?
- Bert or torch from scratch?
- which training methods exactly?
- paper recommendations?
- keywords fo googling?

In [1]:
from transformers import BertTokenizer, BertModel
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

text = "BERT embeddings are fascinating."
inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, add_special_tokens=True)
outputs = model(**inputs)

word_embeddings = outputs.last_hidden_state
print(word_embeddings)

c:\Users\csms\miniconda3\envs\NLP-Lecture\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\csms\miniconda3\envs\NLP-Lecture\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\csms\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activat

tensor([[[-0.1094,  0.2831, -0.2219,  ..., -0.4116,  0.2604,  0.4303],
         [ 0.3717,  0.0461,  0.3014,  ..., -0.2603, -0.2263,  0.0782],
         [-0.2072,  0.3525, -0.2668,  ..., -0.0065, -0.6920, -0.0067],
         ...,
         [ 0.5834,  0.4004,  0.4881,  ..., -0.1181,  0.2630, -0.1225],
         [ 0.0716, -0.2377, -0.3874,  ...,  0.2981,  0.0436, -0.6352],
         [ 0.8747,  0.4112, -0.3423,  ...,  0.2614, -0.4867, -0.2898]]],
       grad_fn=<NativeLayerNormBackward0>)


c:\Users\csms\miniconda3\envs\NLP-Lecture\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [2]:
inputs

{'input_ids': tensor([[  101, 14324,  7861,  8270,  4667,  2015,  2024, 17160,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [3]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

original_text = "Long text for summarization..."
inputs = tokenizer(original_text, return_tensors='pt', padding=True, truncation=True)

summary_logits = model(**inputs).logits
summary = tokenizer.decode(torch.argmax(summary_logits, dim=1))
print("Summary:", summary)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Summary: [unused0]


In [5]:
from transformers import BertForSequenceClassification, BertTokenizer
import torch

model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

text = "Sample text for training."
label = 1  # Assuming positive sentiment

inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
outputs = model(**inputs, labels=torch.tensor([label]))

loss = outputs.loss
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
loss.backward()
optimizer.step()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\csms\miniconda3\envs\NLP-Lecture\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
